In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS_NUM_THREADS"] = "1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] = "1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] = "1" # export NUMEXPR_NUM_THREADS=1

import csv
from IPython.display import display
import sys
import time
import pandas as pd
import numpy as np
import tqdm
import shap

import matplotlib.pyplot as plt
import sklearn
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV, PredefinedSplit, cross_validate, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay
from sklearn.multioutput import MultiOutputClassifier
import imblearn
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import xgboost as xgb


In [ ]:
%run ml_functions.py

In [ ]:
demo_train, demo_test=[],[]
cond_train, cond_test=[],[]
util_train, util_test=[],[]
lab_cts_train, lab_cts_test=[],[]
lab_train, lab_test=[],[]
drug_train,drug_test=[],[]
proc_train, proc_test=[],[]

In [ ]:
demo_all=pd.read_csv(make_path("demo","","all"), index_col=[0]).drop(columns=['index_visit_occurrence_id', 'birth_date', 'index_month', 'index_year', 'index_date', 'observation_type'])
cond_all=pd.read_csv(make_path("cond","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])
util_all=pd.read_csv(make_path("util","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])
lab_cts_all=pd.read_csv(make_path("lab_cts","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])
drug_all=pd.read_csv(make_path("drug","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])
proc_all=pd.read_csv(make_path("proc","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])
lab_all=pd.read_csv(make_path("lab","", "all"), index_col=[0]).drop(columns=['index_date', 'pasc_flag'])

In [ ]:
feature_df_all=make_feature_df(demo_all, cond_all, drug_all, util_all, lab_all, proc_all, lab_cts_all)

In [ ]:
X_train, y_train=feature_df_all[0], feature_df_all[1]
y_train_multiclass=(y_train[:,0]+y_train[:,1])
cols=X_train.columns.values
dim=X_train.shape[1]
ct_scale=ColumnTransformer([('scale', StandardScaler(), cols)], remainder="passthrough") 
X_scaled_train=ct_scale.fit_transform(X_train)
xgb_estimator = xgb.XGBClassifier(n_estimators=150, learning_rate=0.1, colsample_bytree=0.7, objective='multi:softprob', num_class=3, n_jobs=16)
xgb_estimator.fit(X_scaled_train, y_train_multiclass)

In [ ]:
pd.DataFrame(X_scaled_train).to_csv("../specs/X_scaled_all.csv", index=False)

In [ ]:
lab_all.shape

In [ ]:
pd.DataFrame(X_train.columns).to_csv("../specs/X_test_columns.csv", index=False)
pd.DataFrame(X_train.columns[:23]).to_csv("../specs/demo_columns.csv", index=False)
pd.DataFrame(cond_all.columns).to_csv("../specs/cond_columns.csv", index=False)
pd.DataFrame(util_all.columns).to_csv("../specs/util_columns.csv", index=False)
pd.DataFrame(lab_cts_all.columns).to_csv("../specs/lab_cts_columns.csv", index=False)
pd.DataFrame(drug_all.columns).to_csv("../specs/drug_columns.csv", index=False)
pd.DataFrame(lab_all.columns).to_csv("../specs/lab_columns.csv", index=False)
pd.DataFrame(proc_all.columns).to_csv("../specs/proc_columns.csv", index=False)

In [ ]:
xgb_estimator.save_model("../specs/xgb_model_all_11_12_2022.json")